# CPRN CREATE : PHASE A1 : RRL PROCESSING (OSM Guizhou)
**OSM 道路数据处理（贵州省）**

`version` : `4.1.9`   
`author` : seika <seika@live.ca>   
`create_date` : `2025-11-17`   
`last_update` : `2025-11-20`   

---

## Notes

+ CPRN构建的第一阶段，用于将 OSM 标精地图数据进行清洗、标准化和拓扑修复，生成可用于路网构建的标准化道路数据
+ 输入：OSM 数据（shp/gpkg 格式）
+ 输出：标准化道路数据，位于 `PATH_SQLITE_CPRN` (spatialite) 文件中，以 `LAYER_NAME_RRL_OSM` 图层存储
+ 道路类型筛选：`motorway` + `motorway_link`
+ 连通性修复：根据实际数据问题渐进明细

---


## 0.环境配置


In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

# 基础库
import pandas as pd
import numpy as np
import geopandas as gpd
import fiona
from loguru import logger

# 自定义模块
from taisl_sop.model.gdf.geometry import create_multi_crs_geometry
from taisl_sop.model.junction.junc_proc import extract_gdf_end_points, sjoin_ep_rrl, cut_gdf_rrl_pt

## 0.路径配置

In [2]:
# --- SETTINGS ---
## PARAM: CRS
CRS_CGCS2 = 'epsg:4545'  # CGCS2000 3度带, 中心点为重庆东北方位
CRS_WGS84 = 'epsg:4326'

## EXTENSIONS
PATH_SQLITE_EXT : str = r'/opt/homebrew/Cellar/libspatialite/5.1.0/lib/mod_spatialite.dylib'

## 🌀 INPUTS - OSM 数据路径（待配置）
PATH_OSM_DATA : str = r"/Users/seika/sync/lab_works/ZSTORE/geodata/osm/2511/guizhou-251119-free/gis_osm_roads_free_1.shp"  # OSM shp/gpkg 文件路径

## 💫 OUTPUTS
PATH_SQLITE_CPRN : str = r'/Users/seika/sync/lab_works/ZSTORE/geodata/spatialite/CPRN2505.sqlite'
LAYER_NAME_RRL_OSM : str = 'cprn_roads_osm_guizhou_v419'

## 道路类型筛选规则
HIGHWAY_TYPES = ['motorway', 'motorway_link',]
# HIGHWAY_TYPES = ['motorway', 'motorway_link', 'trunk', 'trunk_link',]

logger.info(f"✅ 配置加载完成")
logger.info(f"   输入数据: {PATH_OSM_DATA}")
logger.info(f"   输出表: {LAYER_NAME_RRL_OSM}")
logger.info(f"   道路类型: {HIGHWAY_TYPES}")


2025-11-20 09:58:06.203 | INFO     | __main__:<module>:20 - ✅ 配置加载完成
2025-11-20 09:58:06.203 | INFO     | __main__:<module>:21 -    输入数据: /Users/seika/sync/lab_works/ZSTORE/geodata/osm/2511/guizhou-251119-free/gis_osm_roads_free_1.shp
2025-11-20 09:58:06.203 | INFO     | __main__:<module>:22 -    输出表: cprn_roads_osm_guizhou_v419
2025-11-20 09:58:06.204 | INFO     | __main__:<module>:23 -    道路类型: ['motorway', 'motorway_link']


## 1. 数据加载


In [3]:
# 加载 OSM 数据
# 支持 shp 和 gpkg 格式

if PATH_OSM_DATA.endswith('.gpkg'):
    # GPKG 格式：可能有多个图层
    lst_layers = fiona.listlayers(PATH_OSM_DATA)
    logger.info(f"GPKG 文件包含图层: {lst_layers}")
    
    # 读取所有图层并合并（或指定特定图层）
    gdf_osm = gpd.GeoDataFrame()
    for lay in lst_layers:
        gdf_ = gpd.read_file(PATH_OSM_DATA, layer=lay)
        gdf_osm = pd.concat([gdf_osm, gdf_], axis=0, ignore_index=True)
        logger.info(f"  图层 {lay}: {len(gdf_)} 条记录")
else:
    # SHP 格式：直接读取
    gdf_osm = gpd.read_file(PATH_OSM_DATA)
    logger.info(f"SHP 文件: {len(gdf_osm)} 条记录")

logger.info(f"✅ OSM 数据加载完成: {len(gdf_osm)} 条记录")
logger.info(f"   字段列表: {gdf_osm.columns.tolist()}")
gdf_osm.head(3)

2025-11-20 09:58:06.888 | INFO     | __main__:<module>:18 - SHP 文件: 124289 条记录
2025-11-20 09:58:06.889 | INFO     | __main__:<module>:20 - ✅ OSM 数据加载完成: 124289 条记录
2025-11-20 09:58:06.889 | INFO     | __main__:<module>:21 -    字段列表: ['osm_id', 'code', 'fclass', 'name', 'ref', 'oneway', 'maxspeed', 'layer', 'bridge', 'tunnel', 'geometry']


,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,geometry
0,39482054,5111,motorway,惠兴高速,S50,F,0,0,F,F,"LINESTRING (105.29064 25.4738, 105.28963 25.47..."
1,39482059,5113,primary,宝山北路,None,F,0,0,F,F,"LINESTRING (106.71468 26.59578, 106.71465 26.5..."
2,39482060,5112,trunk,关兴公路,S213,B,0,0,F,F,"LINESTRING (105.68676 25.62986, 105.68695 25.6..."


## 2. 道路类型筛选


In [4]:
gdf_rrl = gdf_osm.query(f"fclass in {HIGHWAY_TYPES}").copy()

logger.info(f"✅  道路类型筛选完成")
logger.info(f"  筛选前: {len(gdf_osm)} 条记录")
logger.info(f"  筛选后: {len(gdf_rrl)} 条记录")
logger.info(f"  筛选率: {len(gdf_rrl)/len(gdf_osm)*100:.2f}%")

2025-11-20 09:58:06.923 | INFO     | __main__:<module>:3 - ✅  道路类型筛选完成
2025-11-20 09:58:06.923 | INFO     | __main__:<module>:4 -   筛选前: 124289 条记录
2025-11-20 09:58:06.923 | INFO     | __main__:<module>:5 -   筛选后: 31476 条记录
2025-11-20 09:58:06.924 | INFO     | __main__:<module>:6 -   筛选率: 25.32%


## 3. 字段映射与标准化

In [5]:
# 字段重命名
gdf_rrl = gdf_rrl.rename(columns={
    'name': 'rrl_name',
    'ref': 'road_code',
})

# 填充空值
gdf_rrl['road_code'] = gdf_rrl['road_code'].fillna('')
gdf_rrl['rrl_name'] = gdf_rrl['rrl_name'].fillna('')

# 字段值处理
# bridge/tunnel: 先填充空值，再映射
gdf_rrl['bridge'] = gdf_rrl['bridge'].fillna('F').map({'T': 'BDG', 'F': ''})
gdf_rrl['tunnel'] = gdf_rrl['tunnel'].fillna('F').map({'T': 'TUN', 'F': ''})

# rtype: 根据 fclass 映射
gdf_rrl['rtype'] = gdf_rrl['fclass'].map({
    'motorway': 'NA', 
    'motorway_link': 'ramp',
    'trunk': 'NA', 
    'trunk_link': 'ramp'
})

# cls: 组合 bridge 和 tunnel，然后根据 fclass 补充默认值
gdf_rrl['cls'] = gdf_rrl['bridge'] + gdf_rrl['tunnel']
# 如果 cls 为空且 fclass 为 motorway，则赋值为 'MR'
cls_mask_mr = (gdf_rrl['cls'] == '') & (gdf_rrl['fclass'] == 'motorway')
gdf_rrl.loc[cls_mask_mr, 'cls'] = 'MR'
# 如果 cls 为空且 fclass 为 trunk，则赋值为 'TR'
cls_mask_tr = (gdf_rrl['cls'] == '') & (gdf_rrl['fclass'] == 'trunk')
gdf_rrl.loc[cls_mask_tr, 'cls'] = 'TR'

# 选择输出字段
cols_output = ['osm_id', 'road_code', 'rrl_name', 'cls', 'rtype', 'maxspeed', 'oneway', 'layer', 'geometry']
gdf_rrl = gdf_rrl[cols_output].copy()

## 道路几何操作

In [6]:
# 炸开 MultiLineString（如果存在）
logger.info(f"处理前: {len(gdf_rrl)} 条记录")
geom_type_counts = gdf_rrl.geometry.geom_type.value_counts()
logger.info(f"几何类型分布:")
logger.info(f"{geom_type_counts}")

# 检查是否有 MultiLineString，如果有则炸开
if 'MultiLineString' in geom_type_counts.index:
    logger.info("发现 MultiLineString，执行炸开操作...")
    gdf_rrl = gdf_rrl.explode(index_parts=False).reset_index(drop=True)
    logger.info(f"处理后: {len(gdf_rrl)} 条记录")
    logger.info(f"几何类型分布:")
    logger.info(f"{gdf_rrl.geometry.geom_type.value_counts()}")
else:
    logger.info("未发现 MultiLineString，跳过炸开操作")

2025-11-20 09:58:06.975 | INFO     | __main__:<module>:2 - 处理前: 31476 条记录
2025-11-20 09:58:06.979 | INFO     | __main__:<module>:4 - 几何类型分布:
2025-11-20 09:58:06.980 | INFO     | __main__:<module>:5 - LineString    31476
Name: count, dtype: int64
2025-11-20 09:58:06.980 | INFO     | __main__:<module>:15 - 未发现 MultiLineString，跳过炸开操作


In [7]:
# 创建多坐标系几何
# WGS84 + CGCS2000

dct_crs = {'geom_rrl_wgs': CRS_WGS84, 'geom_rrl_cgcs': CRS_CGCS2}
gdf_rrl = create_multi_crs_geometry(
    gdf=gdf_rrl,
    geom_col='geometry',
    crs= dct_crs,
    keep_original=False,
    verbose=True
)

logger.info(f"✅ 多坐标系几何创建完成,\n {dct_crs}")
gdf_rrl.sample(4)

2025-11-20 09:58:07.114 | INFO     | __main__:<module>:13 - ✅ 多坐标系几何创建完成,
 {'geom_rrl_wgs': 'epsg:4326', 'geom_rrl_cgcs': 'epsg:4545'}


,osm_id,road_code,rrl_name,cls,rtype,maxspeed,oneway,layer,geom_rrl_wgs,geom_rrl_cgcs
12368,442214954,,,,ramp,0,F,0,"LINESTRING (105.02989 26.57724, 105.03023 26.5...","LINESTRING (204027.594 2944222.12, 204060.905 ..."
73103,941655168,S61,赫六高速,TUN,NA,0,F,-1,"LINESTRING (104.81348 27.13051, 104.81341 27.1...","LINESTRING (183992.526 3006101.115, 183986.638..."
3857,260504115,G76,厦蓉高速,BDG,NA,0,F,1,"LINESTRING (108.62309 25.87102, 108.6249 25.87...","LINESTRING (562451.087 2862693.774, 562632.951..."
82930,1026791308,G76,厦蓉高速,BDG,NA,0,F,1,"LINESTRING (105.39809 27.52812, 105.39808 27.5...","LINESTRING (242914.09 3048849.181, 242912.832 ..."


## 5. 连通性修复


In [8]:
# RRL连通性修复(道路端点+控制点空间分析与断开)
# 参考江苏 A1 步骤 3 的实现

logger.info("=" * 60)
logger.info("开始连通性修复")
logger.info("=" * 60)

# 准备工作：添加索引列
gdf_rrl_cf = gdf_rrl.copy()
gdf_rrl_cf = gdf_rrl_cf.sort_values(['osm_id']).reset_index(drop=True)
gdf_rrl_cf['idx_rrl'] = gdf_rrl_cf.index
logger.info(f"   处理前记录数: {len(gdf_rrl_cf)}")

# 提取道路端点表
logger.info("提取道路端点...")
gdf_rrl_ep = extract_gdf_end_points(gdf_rrl_cf, geom_col='geom_rrl_cgcs')
gdf_rrl_ep = create_multi_crs_geometry(
    gdf=gdf_rrl_ep, 
    geom_col='geom', 
    crs={'geom_ep_wgs': CRS_WGS84, 'geom_ep_cgcs': CRS_CGCS2}, 
    keep_original=False, 
    verbose=False
)
logger.info(f"   提取端点数: {len(gdf_rrl_ep)}")

# 使用 rrl ep 找出需要断开的 rrl (及其 cut point)
logger.info("空间连接分析：端点与道路...")
cols_rrl_ep = ['idx_ep', 'geom_ep_wgs', 'geom_ep_cgcs']
cols_rrl = ['idx_rrl', 'geom_rrl_wgs', 'geom_rrl_cgcs']
cols_sj_eprrl = ['idx_ep', 'idx_rrl', 'de9im_pt_line', 'de9im_line_pt', 
                 'geom_ep_wgs', 'geom_ep_cgcs', 'geom_rrl_wgs', 'geom_rrl_cgcs']

gdf_sj_eprrl = sjoin_ep_rrl(
    gdf_rrl_ep, gdf_rrl_cf, 
    cols_ep=cols_rrl_ep, 
    cols_rrl=cols_rrl, 
    cols_sj=cols_sj_eprrl,
    crs_name_cgcs=CRS_CGCS2
)
logger.info(f"   空间连接结果: {len(gdf_sj_eprrl)} 条记录")

# 筛选需要断开的点：端点在线段内部（非端点）
# de9im_pt_line == '0FFFFF102' 表示点在线上（内部）
# de9im_line_pt == '0F1FF0FF2' 表示线包含点（内部）
gdf_sj_eprrl_cut = gdf_sj_eprrl.query(
    "de9im_pt_line == '0FFFFF102' and de9im_line_pt == '0F1FF0FF2'"
)
logger.info(f"   需要断开的点: {len(gdf_sj_eprrl_cut)} 个")

# 在断点处断开道路
if len(gdf_sj_eprrl_cut) > 0:
    logger.info("执行道路断开...")
    col_base_rrl = ['idx_rrl', 'osm_id', 'road_code', 'rrl_name', 
                    'cls', 'rtype', 'maxspeed', 'oneway', 'layer']
    
    gdf_rrl_cf = cut_gdf_rrl_pt(
        gdf_rrl_cf, 
        gdf_sj_eprrl_cut,
        col_idx='idx_rrl',
        col_geom_rrl='geom_rrl_cgcs',
        col_geom_pt='geom_ep_cgcs',
        cols_rrl=col_base_rrl,
        crs_cut=CRS_CGCS2,
        verbose=True
    )
    
    # 重新创建多坐标系几何（因为断开后几何已改变）
    logger.info("重新创建多坐标系几何...")
    gdf_rrl_cf = create_multi_crs_geometry(
        gdf=gdf_rrl_cf,
        geom_col='geom',
        crs={'geom_rrl_wgs': CRS_WGS84, 'geom_rrl_cgcs': CRS_CGCS2},
        keep_original=False,
        verbose=True
    )
    
    gdf_rrl_cf = gdf_rrl_cf.reset_index(drop=True)
    
    logger.info(f"   断开后记录数: {len(gdf_rrl_cf)}")
    logger.info(f"   新增记录数: {len(gdf_rrl_cf) - len(gdf_rrl)}")
else:
    logger.info("   无需断开的点，跳过断开操作")

# 更新 gdf_rrl（移除临时索引列 idx_rrl）
gdf_rrl = gdf_rrl_cf.drop(columns=['idx_rrl'], errors='ignore').copy()

logger.info("=" * 60)
logger.info("✅ 连通性修复完成")
logger.info("=" * 60)

2025-11-20 09:58:07.133 | INFO     | __main__:<module>:4 - ============================================================
2025-11-20 09:58:07.133 | INFO     | __main__:<module>:5 - 开始连通性修复
2025-11-20 09:58:07.134 | INFO     | __main__:<module>:6 - ============================================================
2025-11-20 09:58:07.151 | INFO     | __main__:<module>:12 -    处理前记录数: 31476
2025-11-20 09:58:07.152 | INFO     | __main__:<module>:15 - 提取道路端点...
2025-11-20 09:58:07.757 | INFO     | __main__:<module>:24 -    提取端点数: 62952
2025-11-20 09:58:07.757 | INFO     | __main__:<module>:27 - 空间连接分析：端点与道路...
2025-11-20 09:58:09.028 | INFO     | __main__:<module>:40 -    空间连接结果: 128681 条记录
2025-11-20 09:58:09.035 | INFO     | __main__:<module>:48 -    需要断开的点: 4371 个
2025-11-20 09:58:09.035 | INFO     | __main__:<module>:52 - 执行道路断开...
2025-11-20 09:58:09.042 | INFO     | taisl_sop.model.junction.junc_proc:split_gdf_rrl:99 - len gdf_rrl:31476;3650/27826
2025-11-20 09:58:10.502 | INFO     | __main_

In [9]:
gdf_rrl_cf

,idx_rrl,osm_id,road_code,rrl_name,cls,rtype,maxspeed,oneway,layer,geom_rrl_wgs,geom_rrl_cgcs
0,2,1000800926,,,,ramp,0,F,0,"LINESTRING (108.0205 27.62318, 108.01984 27.62...","LINESTRING (502023.034 3056684.304, 501958.222..."
1,2,1000800926,,,,ramp,0,F,0,"LINESTRING (108.01957 27.62416, 108.01813 27.6...","LINESTRING (501931.48 3056792.609, 501788.97 3..."
2,3,1000800927,,,,ramp,0,F,0,"LINESTRING (108.01599 27.62601, 108.01712 27.6...","LINESTRING (501578.489 3056997.209, 501690.163..."
3,3,1000800927,,,,ramp,0,F,0,"LINESTRING (108.01905 27.62445, 108.01982 27.6...","LINESTRING (501880.417 3056824.493, 501956.199..."
4,38,1000811151,G69;S10,银百高速,MR,NA,0,F,0,"LINESTRING (107.44854 28.62351, 107.44794 28.6...","LINESTRING (446072.538 3167666.092, 446013.606..."
...,...,...,...,...,...,...,...,...,...,...,...
35823,31471,998768400,G60,沪昆高速,BDG,NA,0,F,1,"LINESTRING (104.42687 25.73848, 104.42858 25.7...","LINESTRING (141332.283 2852722.714, 141503.461..."
35824,31472,998768407,G60,沪昆高速,BDG,NA,0,F,1,"LINESTRING (104.40631 25.73456, 104.40805 25.7...","LINESTRING (139255.077 2852343.663, 139430.361..."
35825,31473,998768408,G60,沪昆高速,BDG,NA,0,F,1,"LINESTRING (104.40806 25.73486, 104.40631 25.7...","LINESTRING (139431.141 2852372.397, 139255.583..."
35826,31474,998768409,G60,沪昆高速,MR,NA,0,F,0,"LINESTRING (104.40533 25.73443, 104.40631 25.7...","LINESTRING (139155.993 2852332.579, 139255.077..."


## 6. 数据质量检查
bypass

In [10]:
gdf_rrl.sample(4)

,osm_id,road_code,rrl_name,cls,rtype,maxspeed,oneway,layer,geom_rrl_wgs,geom_rrl_cgcs
9557,1083432515,G69,银百高速,BDG,NA,0,F,1,"LINESTRING (106.71332 26.28862, 106.71339 26.2...","LINESTRING (371490.786 2909449.675, 371497.392..."
30231,733594489,S32,玉新高速,BDG,NA,0,F,1,"LINESTRING (108.04979 27.63394, 108.04841 27.6...","LINESTRING (504913.503 3057877.501, 504777.985..."
15945,1433755374,,,,ramp,0,F,1,"LINESTRING (105.59114 25.785, 105.59199 25.785...","LINESTRING (258348.502 2855226.104, 258433.69 ..."
33881,946403784,G7611,都香高速,BDG,NA,0,F,1,"LINESTRING (104.90462 26.62151, 104.90537 26.6...","LINESTRING (191655.65 2949428.073, 191730.378 ..."


## 7. 持久化与取回

In [11]:
# 🌖 持久化
# 准备输出字段（CPRN 标准字段）
cols_output = ['osm_id', 'road_code', 'rrl_name', 
               'cls', 'rtype', 'maxspeed', 'oneway', 'layer',
               'geom_rrl_wgs']

# 确保所有字段存在
cols_available = [col for col in cols_output if col in gdf_rrl.columns]
gdf_rrl_output = gdf_rrl[cols_available].copy()

# 设置几何字段为 geom_rrl_wgs（用于持久化）
gdf_rrl_output = gdf_rrl_output.set_geometry('geom_rrl_wgs')
gdf_rrl_output = gdf_rrl_output.set_crs(CRS_WGS84)

# 检查几何有效性
n_invalid = (~gdf_rrl_output.geometry.is_valid).sum()
if n_invalid > 0:
    logger.warning(f"发现 {n_invalid} 条无效几何，将在持久化前移除")
    gdf_rrl_output = gdf_rrl_output[gdf_rrl_output.geometry.is_valid].copy()

n_empty = gdf_rrl_output.geometry.is_empty.sum()
if n_empty > 0:
    logger.warning(f"发现 {n_empty} 条空几何，将在持久化前移除")
    gdf_rrl_output = gdf_rrl_output[~gdf_rrl_output.geometry.is_empty].copy()

# 持久化到 Spatialite
logger.info(f"🌕 持久化到: {LAYER_NAME_RRL_OSM}")
gdf_rrl_output.to_file(
    PATH_SQLITE_CPRN, 
    driver='SQLite', 
    spatialite=True,
    layer=LAYER_NAME_RRL_OSM, 
    overwrite=True, 
    index=False,
    mode='w', 
    encoding='utf-8'
)

logger.info(f"✅ 数据持久化完成")
logger.info(f"   表名: {LAYER_NAME_RRL_OSM}")
logger.info(f"   记录数: {len(gdf_rrl_output)}")
logger.info(f"   字段: {cols_available}")

# 显示样本数据
gdf_rrl_output.sample(4)

2025-11-20 09:58:10.645 | INFO     | __main__:<module>:27 - 🌕 持久化到: cprn_roads_osm_guizhou_v419
2025-11-20 09:58:11.135 | INFO     | __main__:<module>:39 - ✅ 数据持久化完成
2025-11-20 09:58:11.136 | INFO     | __main__:<module>:40 -    表名: cprn_roads_osm_guizhou_v419
2025-11-20 09:58:11.136 | INFO     | __main__:<module>:41 -    记录数: 35828
2025-11-20 09:58:11.136 | INFO     | __main__:<module>:42 -    字段: ['osm_id', 'road_code', 'rrl_name', 'cls', 'rtype', 'maxspeed', 'oneway', 'layer', 'geom_rrl_wgs']


,osm_id,road_code,rrl_name,cls,rtype,maxspeed,oneway,layer,geom_rrl_wgs
33775,946166044,S55,仁望高速,MR,NA,0,F,0,"LINESTRING (106.12116 25.62525, 106.12283 25.6..."
4600,574492788,,,,ramp,0,B,0,"LINESTRING (108.89459 27.23387, 108.89473 27.2..."
8094,1000830201,S10,印习高速,MR,NA,0,F,0,"LINESTRING (107.05245 28.74164, 107.05116 28.7..."
2235,301556446,S50,惠兴高速,MR,NA,0,F,0,"LINESTRING (106.50955 26.03569, 106.51223 26.0..."


In [12]:
# 🌞 取回
from taisl_sop.data.base.spatialite import SpatialitePuller

sp = SpatialitePuller(
    path_sqlite=PATH_SQLITE_CPRN, 
    path_spatialite_ext=PATH_SQLITE_EXT, 
    set_precision=True, 
    decimal_precision=16, 
    driver='sqlite', 
    verbose=True
)

# 构建 SQL 查询
sql = f"""
SELECT ogc_fid, osm_id, road_code, rrl_name, 
       cls, rtype, maxspeed, oneway, layer,
       ASText(GEOMETRY) AS wkt,
       UPPER(ST_GeoHash(GEOMETRY, 12)) AS rrl_geohash
  FROM {LAYER_NAME_RRL_OSM};
"""

logger.info(f"🌞 从数据库取回: {LAYER_NAME_RRL_OSM}")
gdf_rrl_retrieved = sp.query_sql_gdf(sql, col_geom=['wkt'])
gdf_rrl_retrieved = gdf_rrl_retrieved.rename_geometry('geom_rrl_wgs')
gdf_rrl_retrieved = gdf_rrl_retrieved.set_crs(CRS_WGS84)

logger.info(f"✅ 数据取回完成")
logger.info(f"   表名: {LAYER_NAME_RRL_OSM}")
logger.info(f"   记录数: {len(gdf_rrl_retrieved)}")
logger.info(f"   字段: {gdf_rrl_retrieved.columns.tolist()}")

# 显示样本数据
gdf_rrl_retrieved.sample(4)


2025-11-20 09:58:11.153 | INFO     | taisl_sop.data.base.spatialite:__init__:42 - SpatialitePuller initiated with path_sqlite: /Users/seika/sync/lab_works/ZSTORE/geodata/spatialite/CPRN2505.sqlite
2025-11-20 09:58:11.153 | INFO     | taisl_sop.data.base.spatialite:__init__:43 - Path_spatialite_ext: /opt/homebrew/Cellar/libspatialite/5.1.0/lib/mod_spatialite.dylib
2025-11-20 09:58:11.153 | INFO     | taisl_sop.data.base.spatialite:__init__:45 - Set_precision: ✅, decimal_precision: 16
2025-11-20 09:58:11.154 | INFO     | __main__:<module>:22 - 🌞 从数据库取回: cprn_roads_osm_guizhou_v419


2025-11-20 09:58:11.276 | INFO     | taisl_sop.data.base.spatialite:_setup_precision:83 - SpatiaLite version: 5.1.0
2025-11-20 09:58:11.277 | INFO     | taisl_sop.data.base.spatialite:_setup_precision:92 - ✅ Decimal precision set to: 16
2025-11-20 09:58:11.689 | INFO     | taisl_sop.data.geometry.handler:_handle_multiple_geometries:98 - Processed geometry columns: ['geom_wkt']
2025-11-20 09:58:11.690 | INFO     | taisl_sop.data.geometry.handler:_handle_multiple_geometries:99 - Active geometry column: geom_wkt
2025-11-20 09:58:11.702 | INFO     | __main__:<module>:27 - ✅ 数据取回完成
2025-11-20 09:58:11.703 | INFO     | __main__:<module>:28 -    表名: cprn_roads_osm_guizhou_v419
2025-11-20 09:58:11.703 | INFO     | __main__:<module>:29 -    记录数: 35828
2025-11-20 09:58:11.703 | INFO     | __main__:<module>:30 -    字段: ['ogc_fid', 'osm_id', 'road_code', 'rrl_name', 'cls', 'rtype', 'maxspeed', 'oneway', 'layer', 'wkt', 'rrl_geohash', 'geom_rrl_wgs']


Memory usage of df puller : gdf 3.28 MB, df None MB


,ogc_fid,osm_id,road_code,rrl_name,cls,rtype,maxspeed,oneway,layer,wkt,rrl_geohash,geom_rrl_wgs
16514,16515,230614871,G69,,,ramp,0,F,0,"LINESTRING(106.7387851 26.467041, 106.7388573 ...",WKEYSFHBNTMU,"LINESTRING (106.73879 26.46704, 106.73886 26.4..."
220,221,1063737534,,,,ramp,0,F,0,"LINESTRING(108.9197848 25.9403706, 108.9209381...",WKT7WD6QC3T3,"LINESTRING (108.91978 25.94037, 108.92094 25.9..."
9150,9151,1065814456,G56,杭瑞高速,MR,NA,0,F,0,"LINESTRING(108.2968268 27.9603353, 108.295978 ...",WKVP0651CJ6R,"LINESTRING (108.29683 27.96034, 108.29598 27.9..."
1511,1512,1312644734,,,,ramp,0,F,0,"LINESTRING(107.216411 28.0066711, 107.2156946 ...",WKUPRDPTUN6M,"LINESTRING (107.21641 28.00667, 107.21569 28.0..."


In [13]:
gdf_rrl_retrieved.groupby(['oneway']).size()

oneway
B      304
F    35512
T       12
dtype: int64